In [1]:
import os
import zipfile
from movieRecommender.logging import logger


In [2]:
%pwd

'e:\\PROJECTS\\ML\\Movie-Recommender\\expriments'

In [3]:
%cd ..

e:\PROJECTS\ML\Movie-Recommender


In [4]:
%pwd

'e:\\PROJECTS\\ML\\Movie-Recommender'

In [35]:
import os
import zipfile
import requests
from pathlib import Path
from movieRecommender.logging import logger
from dataclasses import dataclass

@dataclass(frozen=True)
class DataDownloader:
    filepath: Path
    extract_path: Path 
    url: str

    def download_file(self) -> None:
        """
        Downloads a file from a URL and saves it to the local file system.
        
        Args:
            url (str): The URL of the file to download.
            file_path (str): The local file path to save the downloaded file.
            logger (logging.Logger): A logger object to log the progress.
        
        Returns:
            None
        """
        try:
            file_path = os.path.join(os.getcwd(), self.filepath, 'file.zip')
            os.makedirs(self.filepath, exist_ok=True)
            response = requests.get(self.url, stream=True)
            total_size = int(response.headers.get('content-length', 0))
            block_size = 1024
            downloaded = 0

            with open(file_path, 'wb') as file:
                for chunk in response.iter_content(chunk_size=block_size):
                    if chunk:
                        file.write(chunk)
                        downloaded += len(chunk)
                        progress = round((downloaded / total_size) * 100, 2)
                        logger.info(f'Downloaded {progress}% of the file')

            logger.info('Download complete!')
        except Exception as e:
            logger.error(f'Error downloading file: {e}')
            return e

            

    def unzip_file(self):
        """
        Extracts the contents of a ZIP file to the specified directory.
        
        Args:
            file_path (str): The path to the ZIP file.
            extract_path (str): The directory where the contents will be extracted.
        
        Returns:
            None
        """
        try:
            os.makedirs(self.extract_path, exist_ok=True)
            with zipfile.ZipFile(self.filepath + '/file.zip', 'r') as zip_ref:
                zip_ref.extractall(self.extract_path)
            logger.info(f'Extraction complete. Files extracted to {self.extract_path}')
        except Exception as e:
            logger.error(f'Error extracting file: {e}')



In [36]:
from movieRecommender.constants import *
from movieRecommender.utils.common import load_config, create_directories
class ConfigHandler:
    def __init__(self, file_path=CONFIG_FILE_PATH, params_path = PARAMS_FILE_PATH):
        self.config = load_config(file_path)
   

    
    def get_fetch_data_config(self) -> DataDownloader:
        config = self.config.download_config
        fetch_data_config = DataDownloader(
            filepath = config.filepath,
            extract_path= config.extract_path,
            url= config.url
        )
        return fetch_data_config

In [37]:
try:
    config = ConfigHandler()
    fetch_data_config = config.get_fetch_data_config()
    # fetch_data_config.download_file()
    fetch_data_config.unzip_file()
except Exception as e:
    raise e

[2024-05-03 12:00:42,217: INFO: common: Yaml file: config\config.yaml loaded successfully]
[2024-05-03 12:00:43,549: INFO: 2901575274: Extraction complete. Files extracted to artifacts/unzip_data]
